# Task 3:

Proponer un demostrador que capture las imágenes de la cámara, y les permita exhibir lo aprendido en estas dos prácticas ante quienes no cursen la asignatura :). Es por ello que además de poder mostrar la imagen original de la webcam, incluya al menos dos usos diferentes de aplicar las funciones de OpenCV trabajadas hasta ahora.

In [1]:
# Import necessary libraries
import numpy as np
import cv2

In [2]:
def getAndDrawCols(frame):
    try:
        # Convert the input frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply the Canny edge detection algorithm to the grayscale image
        canny_frame = cv2.Canny(gray_frame, 100, 200)

        # Apply a binary threshold to the Canny edge-detected image
        # Pixels above 127 are set to 255 (white), others are set to 0 (black)
        res, threshold_canny_frame = cv2.threshold(canny_frame, 127, 255, cv2.THRESH_BINARY)

        # Sum the pixel values in each column (across all rows)
        # This gives the number of white pixels in each column
        col_counts_frame = cv2.reduce(threshold_canny_frame, 0, cv2.REDUCE_SUM, dtype=cv2.CV_32SC1)

        # Normalize the column sums by dividing by the maximum possible value (255 * number of rows)
        # This gives the proportion of white pixels in each column
        cols_frame = col_counts_frame[0] / (255 * threshold_canny_frame.shape[0])

        # Find the columns where the proportion of white pixels is greater than or equal to 85% 
        # of the maximum proportion in the entire frame
        cols_max_frame = np.where(cols_frame >= 0.85 * np.amax(cols_frame))

        # Get the bottom position of the frame (height of the image)
        bottom = frame.shape[0]

        # If the number of selected columns is less than two-thirds of the total columns,
        # draw vertical white lines in those columns
        if len(cols_max_frame[0]) < frame.shape[1] * 2/3:
            for col in cols_max_frame[0]:
                cv2.line(frame, (col, 0), (col, bottom), (255,255,255), 4)  # Draw a white line of thickness 4

    # Handle any exceptions that may occur during the process
    except Exception as e:
        print(e)

    # Return the modified frame with the drawn columns
    return frame

In [3]:
def getAndDrawRows(frame):
    try:
        # Convert the input frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply the Canny edge detection algorithm to the grayscale image
        canny_frame = cv2.Canny(gray_frame, 100, 200)

        # Apply a binary threshold to the Canny edge-detected image
        # Pixels above 127 are set to 255 (white), others are set to 0 (black)
        res, threshold_canny_frame = cv2.threshold(canny_frame, 127, 255, cv2.THRESH_BINARY)

        # Sum the pixel values in each row (across all columns)
        row_counts_frame = cv2.reduce(threshold_canny_frame, 1, cv2.REDUCE_SUM, dtype=cv2.CV_32SC1)

        # Transpose the result to convert from a list of lists to a flat list
        row_counts_frame = row_counts_frame.transpose()

        # Normalize the row sums by dividing by the maximum possible value (255 * number of columns)
        # This gives the proportion of white pixels in each row
        rows_frame = row_counts_frame[0] / (255 * threshold_canny_frame.shape[1])

        # Find the rows where the proportion of white pixels is greater than or equal to 85%
        # of the maximum proportion in the entire frame
        rows_max_frame = np.where(rows_frame >= 0.85 * np.amax(rows_frame))

        # Get the end position of the frame (width of the image)
        end = frame.shape[1]

        # If the number of selected rows is less than two-thirds of the total rows,
        # draw horizontal white lines in those rows
        if len(rows_max_frame[0]) < frame.shape[0] * 2/3:
            for row in rows_max_frame[0]:
                cv2.line(frame, (0, row), (end, row), (255,255,255), 4)  # Draw a white line of thickness 4

    # Handle any exceptions that may occur during the process
    except Exception as e:
        print(e)

    # Return the modified frame with the drawn rows
    return frame

In [4]:
def pop_art(frame):
    ncells = 8  # Define the size of the cell grid (8x8 pixels per cell)
    
    # Get the original dimensions of the input frame (height, width, channels)
    h, w, c = frame.shape

    # Create a black image with the same dimensions as the original frame
    pop_art = np.zeros((h, w, 3), dtype=np.uint8)

    # Loop through the image in steps of 'ncells' (creating a grid of 8x8 cells)
    for y in range(0, h, ncells):
        for x in range(0, w, ncells):
            # Get the RGB values of the current pixel at (x, y)
            r = frame[y, x, 0]
            g = frame[y, x, 1]
            b = frame[y, x, 2]

            # If red is the dominant color, draw an ellipse with a certain color adjustment
            if r > g and r > b:
                cv2.ellipse(pop_art, (x + int(ncells / 2), y + int(ncells / 2)), 
                            (int(ncells - 1), int(ncells / 2)), 135, 0, 180, 
                            (int(r), int(g / 1.3), int(b / 1.3)), -1)
            # Otherwise, draw an ellipse with different color adjustments
            else:
                cv2.ellipse(pop_art, (x + int(ncells / 2), y + int(ncells / 2)), 
                            (int(ncells - 1), int(ncells / 2)), 45, 0, 180, 
                            (int(r / 1.3), int(g), int(b)), -1)

    # Display the resulting 'pop art' image, resized to fit the screen
    cv2.imshow('Cam', pop_art)

In [5]:
def zona_clara_oscura(frame):
    # Convert the input frame to grayscale
    grayimg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Get the indices of the brightest (max) and darkest (min) pixels in the grayscale image
    max = np.where(grayimg == np.amax(grayimg))
    min = np.where(grayimg == np.amin(grayimg))

    # Assign the coordinates of the darkest and brightest pixels to variables
    # The first occurrence is taken and this works well enough
    dx, dy, bx, by = min[1][0], min[0][0], max[1][0], max[0][0]

    # Draw circles around the darkest and brightest pixels
    # Green circle for the darkest pixel, red circle for the brightest pixel
    cv2.circle(frame, (dx, dy), 5, (0,255,0), 2)
    cv2.circle(frame, (bx, by), 5, (0,0,255), 2)

    # Draw the same circles on the grayscale image
    cv2.circle(grayimg, (dx, dy), 5, (255), 2)  # White circle for darkest pixel
    cv2.circle(grayimg, (bx, by), 5, (0), 2)    # Black circle for brightest pixel

    # Display the frame with the highlighted pixels
    cv2.imshow('Videocam', frame)

In [6]:
def eliminador_fondo(frame, backSub):
    # Apply the background subtractor to the frame, generating a foreground mask
    fgMask = backSub.apply(frame)

    # Display the resulting foreground mask (black-and-white image)
    cv2.imshow('Videocam', fgMask)

In [7]:
def diferencia(frame, prevFrame):
    # Calculate the absolute difference between the current frame and the previous frame
    dif = cv2.absdiff(frame, prevFrame)

    # Display the resulting difference image
    cv2.imshow('Videocam', dif)

In [8]:
def diferencia_con_bordes_verticales(frame, prevFrame):
    # Calculate the absolute difference between the current frame and the previous frame
    dif = cv2.absdiff(frame, prevFrame)

    # Call the getAndDrawCols function to process the difference image and draw vertical lines
    dif = getAndDrawCols(dif)    

    # Display the resulting image with vertical edges highlighted
    cv2.imshow('Videocam', dif)

In [9]:
def bordes_verticales(frame):
    # Call the getAndDrawCols function to process the frame and draw vertical lines
    frame = getAndDrawCols(frame)

    # Display the resulting image with vertical edges highlighted
    cv2.imshow('Videocam', frame)

In [10]:
def bordes_horizontales(frame):
    # Call the getAndDrawRows function to process the frame and draw horizontal lines
    frame = getAndDrawRows(frame)

    # Display the resulting image with horizontal edges highlighted
    cv2.imshow('Videocam', frame)

In [11]:
def bordes_verticales_y_horizontales(frame):
    # Call the getAndDrawCols function to process the frame and draw vertical lines
    frame = getAndDrawCols(frame)

    # Call the getAndDrawRows function to process the frame and draw horizontal lines
    frame = getAndDrawRows(frame)

    # Display the resulting image with both vertical and horizontal edges highlighted
    cv2.imshow('Videocam', frame)

In [12]:
def canny_output(frame):
    # Convert the input frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply the Canny edge detection algorithm to the grayscale image
    canny_frame = cv2.Canny(gray_frame, 100, 200)

    # Display the resulting image with detected edges
    cv2.imshow('Videocam', canny_frame)

In [13]:
def umbralizado(frame):
    # Convert the input frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to the grayscale image
    # Pixels above 127 are set to 255 (white), and others are set to 0 (black)
    res, output = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)

    # Display the resulting binary thresholded image
    cv2.imshow('Videocam', output)

In [14]:
def canny_y_umbralizado(frame):
    # Convert the input frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply the Canny edge detection algorithm to the grayscale image
    canny_frame = cv2.Canny(gray_frame, 100, 200)

    # Apply a binary threshold to the Canny edge-detected image
    # Pixels above 127 are set to 255 (white), and others are set to 0 (black)
    res, threshold_canny_frame = cv2.threshold(canny_frame, 127, 255, cv2.THRESH_BINARY)

    # Display the resulting binary thresholded image after Canny detection
    cv2.imshow('Cam', threshold_canny_frame)

In [15]:
def switch(effect, frame, backSub, prevFrame):
    # Check the value of 'effect' and call the corresponding function
    if effect == 0:
        zona_clara_oscura(frame)  # Process frame to highlight light and dark areas
    elif effect == 1:
        pop_art(frame)  # Apply pop art effect to the frame
    elif effect == 2:
        eliminador_fondo(frame, backSub)  # Remove the background from the frame
    elif effect == 3:
        diferencia(frame, prevFrame)  # Show the difference between the current and previous frames
    elif effect == 4:
        diferencia_con_bordes_verticales(frame, prevFrame)  # Show the difference with vertical edges
    elif effect == 5:
        bordes_verticales(frame)  # Highlight vertical edges in the frame
    elif effect == 6:
        bordes_horizontales(frame)  # Highlight horizontal edges in the frame
    elif effect == 7:
        bordes_verticales_y_horizontales(frame)  # Highlight both vertical and horizontal edges
    elif effect == 8:
        canny_output(frame)  # Apply Canny edge detection to the frame
    elif effect == 9:
        umbralizado(frame)  # Apply binary thresholding to the frame
    elif effect == 10:
        canny_y_umbralizado(frame)  # Apply Canny edge detection followed by thresholding

In [ ]:
# Open a video capture object for the default camera (0)
vid = cv2.VideoCapture(0)

# Create a background subtractor using MOG2 method with specified parameters
backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=True)

# Initialize the effect index
effect = 0

# Define the total number of effects available
number_of_effects = 10

# Initialize the previous frame variable
prevFrame = 0

while(True):
    # Read frame by frame from the video capture
    ret, frame = vid.read()

    if ret:
        # Apply a mirror effect on the input frame
        framem = cv2.flip(frame, 1)
        # Call the switch function to apply the selected effect
        switch(effect, framem, backSub, prevFrame)
        # Update the previous frame to the current processed frame
        prevFrame = framem

    # Use keyboard input for interaction
    key = cv2.waitKey(33)

    # Press ESC key to exit the loop
    if key == 27:
        break
    # Press the right arrow key to show the next effect
    elif key == 39:
        effect += 1
        if effect > number_of_effects: effect = 0  # Reset to 0 if exceeds the number of effects
    # Press the left arrow key to show the previous effect
    elif key == 37:
        effect -= 1
        if effect < 0: effect = number_of_effects  # Reset to the maximum if below 0
    else:
        continue

# Release the video capture object when done
vid.release()

# Destroy all OpenCV windows
cv2.destroyAllWindows()